<a href="https://colab.research.google.com/github/fcoliveira-utfpr/chuva_probabilidade_sh/blob/main/analises_anual_chuva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalando bibliotecas e autorizando Drive

In [18]:
# Limpa a saída do terminal no Google Colab
from google.colab import output
output.clear()
# Instala ou atualiza a bibliotecas
!pip install --upgrade gspread -q
!pip install pymannkendall -q
# Autentica o usuário no Google Colab
from google.colab import auth
auth.authenticate_user()
# Importa a biblioteca para interagir com Google Sheets
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

In [19]:
#importando bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config inlineBackend.figure_formats = ['svg']
import datetime
from datetime import datetime
from datetime import timedelta
import math
from scipy import stats
from scipy.stats import (gamma, norm, ks_2samp, beta, weibull_min, expon, lognorm)
from matplotlib.ticker import FuncFormatter
from scipy.stats import ksone
from sklearn.preprocessing import MinMaxScaler
import pymannkendall as mk
from scipy.stats import linregress

Importando e abrindo planilha do Drive

In [ ]:
#importantando planilha e abrindo***********************************************
nome_da_planilha = 'dados_chuva_oeste'
nome_da_aba = "df_resultados"
spreasheet = gc.open(nome_da_planilha)
worksheet = spreasheet.worksheet(nome_da_aba)
df = pd.DataFrame(worksheet.get_all_values())
head = df.loc[0].to_list()
df.columns = head
df = df[1:]
cidade = df['Municipio']
df = df.drop(columns=['Municipio'])
df = df.replace({',': '.'}, regex=True)
df = df.apply(lambda x: pd.to_numeric(x, errors = 'coerce'), axis=1)
df['Municipio'] = cidade
df['Ano'] = df['Ano'].astype(int)
df['Mês'] = df['Mês'].astype(int)
df['Dia'] = df['Dia'].astype(int)
#filtra município desejado
municipio = "Santa Helena"
df = df.loc[df['Municipio'] == municipio]
# Crie uma nova coluna 'data' a partir das colunas 'dia', 'mês' e 'ano'
data = df['Ano'].astype(str) + '-' + df['Mês'].astype(str) + '-' + df['Dia'].astype(str)
dia = []
for i in data:
  a = pd.to_datetime(datetime.strptime(i, '%Y-%m-%d').date())
  dia.append(a)
df['Data'] = dia
df

##Agrupando chuva anual

In [ ]:
#Precipitação pluvial total anual
df_ano = df.groupby(['Ano']).agg({
    'Chuva (mm)': 'sum'
}).reset_index()
df_ano
#anos = list(set(df_ano['Ano'].tolist()))

##Analisando tendência anual

In [ ]:
# Extrair os anos e os valores de chuva
anos = df_ano['Ano']
chuva = df_ano['Chuva (mm)']

# Regressão linear
slope, intercept, r_value, p_value, std_err = linregress(anos, chuva)

# Teste de Mann-Kendall
mk_result = mk.original_test(chuva)

# Armazenar os resultados
result = {
    'Slope': slope,
    'Intercept': intercept,
    'p-value Regressão': p_value,
    'R²': r_value**2,
    'Mann-Kendall S': mk_result.s,
    'Mann-Kendall p-value': mk_result.p,
    'Mann-Kendall Tendência': mk_result.trend
}

# Criar um DataFrame com os resultados para exibição
df_results = pd.DataFrame([result])

# Exibir os resultados
df_results

##Encontrando FDP e FDA

In [ ]:
#pegando uma série da chuva anual
chuva_anual = df_ano['Chuva (mm)']

#Transformando os dados
for i in chuva_anual:
  i + 1000

# Crie um intervalo de valores para a precipitação
x = np.linspace(0, max(chuva_anual), 1000)

# Distribuição Normal
mu, std = stats.norm.fit(chuva_anual)
pdf_normal = stats.norm.pdf(x, mu, std)
D_normal, p_value_normal = stats.kstest(chuva_anual, 'norm', args=(mu, std))
cdf_normal = stats.norm.cdf(x, mu, std)

# Distribuição Exponencial
loc_exp, scale_exp = stats.expon.fit(chuva_anual)
pdf_exp = stats.expon.pdf(x, loc=loc_exp, scale=scale_exp)
D_exp, p_value_exp = stats.kstest(chuva_anual, 'expon', args=(loc_exp, scale_exp))
cdf_exp = stats.expon.cdf(x, loc=loc_exp, scale=scale_exp)

# Distribuição Gama
shape_gama, loc_gama, scale_gama = stats.gamma.fit(chuva_anual)
pdf_gama = stats.gamma.pdf(x, a=shape_gama, loc=loc_gama, scale=scale_gama)
D_gama, p_value_gama = stats.kstest(chuva_anual, 'gamma', args=(shape_gama, loc_gama, scale_gama))
cdf_gama = stats.gamma.cdf(x, a=shape_gama, loc=loc_gama, scale=scale_gama)

# Distribuição Log-Normal
shape_ln, loc_ln, scale_ln = stats.lognorm.fit(chuva_anual)
pdf_ln = stats.lognorm.pdf(x, s=shape_ln, loc=loc_ln, scale=scale_ln)
D_ln, p_value_ln = stats.kstest(chuva_anual, 'lognorm', args=(shape_ln, loc_ln, scale_ln))
cdf_ln = stats.lognorm.cdf(x, s=shape_ln, loc=loc_ln, scale=scale_ln)

n = len(chuva_anual)
alpha = 0.05  # Nível de significância
D_critical = ksone.ppf(1 - alpha / 2, n)

# Crie um dicionário com os dados
data = {
    "Distribuição": ["Normal", "Exponencial", "Gama", "Log-Normal"],
    "P-valor": [p_value_normal, p_value_exp, p_value_gama, p_value_ln],
    "Dsup": [D_normal, D_exp, D_gama, D_ln],
    "Dcrítico": [D_critical, D_critical, D_critical, D_critical], # Valor de D calculado (Dsup) tem que ser menor que D crítico.
    "Coeficientes": [
            f"mu={mu:.1f}, std={std:.1f}" if not np.isnan(mu) and not np.isnan(std) else "-",
            f"loc={loc_exp:.1f}, scale={scale_exp:.1f}" if not np.isnan(loc_exp) and not np.isnan(scale_exp) else "-",
            f"shape={shape_gama:.1f}, loc={loc_gama:.1f}, scale={scale_gama:.1f}" if not np.isnan(shape_gama) and not np.isnan(loc_gama) and not np.isnan(scale_gama) else "-",
            f"shape={shape_ln:.1f}, loc={loc_ln:.1f}, scale={scale_ln:.1f}" if not np.isnan(shape_ln) and not np.isnan(loc_ln) and not np.isnan(scale_ln) else "-",
        ]
}

# Crie o DataFrame
result = pd.DataFrame(data)

# Determine o melhor ajuste com base no nível de significância
result["Melhor Ajuste"] = "Não siguinificativo"  # Começa com "Inadequado" e é atualizado se um ajuste for adequado

result.loc[result["P-valor"] > alpha, "Melhor Ajuste"] = result.loc[result["P-valor"] > alpha, "Distribuição"]

# Exiba o DataFrame result
result = result.sort_values(by='Dsup').reset_index()
result = result[['Melhor Ajuste', 'P-valor', 'Dsup', 'Dcrítico','Coeficientes']]

for i in chuva_anual:
  i - 1000

result

##Determiando precipitação provável

In [ ]:
nivel_probabilidade = [0.9, 0.8, 0.75, 0.5, 0.25, 0.20, 0.10]
precipitacao_provavel = []

for probabilidade in nivel_probabilidade:
  shape_ln, loc_ln, scale_ln = stats.lognorm.fit(chuva_anual) #usando log normal
  x = np.linspace(0, max(chuva_anual), 1000)
  precipitacao_provavel_anual = max(0, stats.lognorm.ppf(probabilidade, shape_ln, loc_ln, scale_ln))
  precipitacao_provavel.append(precipitacao_provavel_anual)
  # valor de precipitação que é provável de ser igualado ou superado com a probabilidade especificada


# Crie um DataFrame com os resultados
colunas = ["10%", "20%", "25%", "50%", "75%", "80%","90%"]
precipitacao_provavel
df_resultados = pd.DataFrame({
    "Probabilidade": colunas,
    "Chuva provável (mm)": precipitacao_provavel
})
df_resultados